# 03 - Feature Engineering

Ce notebook sélectionne les features importantes et encode les variables catégorielles.

## 1. Import des librairies

In [1]:
# Manipulation de données
import pandas as pd
import numpy as np

# Encodage
from sklearn.preprocessing import LabelEncoder, StandardScaler

print("✓ Librairies importées avec succès")

✓ Librairies importées avec succès


## 2. Chargement des données

In [2]:
# Charger les données brutes
df = pd.read_csv('../data/raw/fraud_synth_10000.csv')

print(f"✓ Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f"Colonnes : {df.columns.tolist()}")

✓ Dataset chargé : 10000 lignes, 9 colonnes
Colonnes : ['transaction_amount', 'transaction_hour', 'num_transactions_24h', 'account_age_days', 'avg_amount_30d', 'country_risk', 'device_type', 'is_foreign_transaction', 'fraud']


## 3. Sélection des features importantes

Basé sur l'analyse, nous gardons uniquement les features avec un impact significatif :

In [3]:
# Liste des features à garder (basé sur l'analyse)
selected_features = [
    'is_foreign_transaction',  # ⭐⭐⭐⭐⭐ Impact +78.5%
    'account_age_days',        # ⭐⭐⭐⭐⭐ Impact -23.9%
    'country_risk',            # ⭐⭐⭐⭐⭐ Impact massif
    'num_transactions_24h',    # ⭐⭐⭐⭐ Impact +15.1%
    'transaction_amount',      # ⭐⭐⭐ Impact +15.3%
    'device_type',             # ⭐⭐⭐ Impact modéré
    'fraud'                    # Variable cible (Y)
]

# Créer un nouveau dataframe avec uniquement les features sélectionnées
df_selected = df[selected_features].copy()

print(f"✓ Features sélectionnées : {len(selected_features) - 1} features + 1 cible (Y)")
print(f"\nFeatures gardées :")
for i, feat in enumerate(selected_features[:-1], 1):
    print(f"  {i}. {feat}")
print(f"  Y. fraud (variable cible)")

✓ Features sélectionnées : 6 features + 1 cible (Y)

Features gardées :
  1. is_foreign_transaction
  2. account_age_days
  3. country_risk
  4. num_transactions_24h
  5. transaction_amount
  6. device_type
  Y. fraud (variable cible)


In [4]:
# Afficher un aperçu des données sélectionnées
print("📊 Aperçu des données sélectionnées :")
df_selected.head(10)

📊 Aperçu des données sélectionnées :


,is_foreign_transaction,account_age_days,country_risk,num_transactions_24h,transaction_amount,device_type,fraud
0,1,3615,low,3,15.01,mobile,0
1,0,209,low,2,24.67,mobile,0
2,0,438,low,4,92.79,mobile,0
3,1,1830,low,6,38.67,desktop,0
4,0,1686,medium,3,69.14,mobile,0
5,1,997,low,4,52.55,mobile,0
6,0,1236,high,1,19.90,mobile,0
7,0,1114,low,4,51.09,mobile,0
8,1,318,high,4,25.71,desktop,0
9,1,3414,low,5,25.59,desktop,0


## 4. Encodage des variables catégorielles

Nous avons 2 variables catégorielles à encoder :
- `country_risk` : 3 catégories (low, medium, high)
- `device_type` : 3 catégories (desktop, mobile, tablet)

### 4.1 One-Hot Encoding pour country_risk

In [5]:
# Afficher les valeurs uniques avant encodage
print("📊 Valeurs de country_risk avant encodage :")
print(df_selected['country_risk'].value_counts())

# Appliquer One-Hot Encoding sur country_risk
# pd.get_dummies crée des colonnes binaires (0/1) pour chaque catégorie
country_risk_encoded = pd.get_dummies(df_selected['country_risk'], prefix='country_risk', drop_first=False)

print(f"\n✓ One-Hot Encoding appliqué sur 'country_risk'")
print(f"Nouvelles colonnes créées : {country_risk_encoded.columns.tolist()}")

# Afficher un aperçu
country_risk_encoded.head()

📊 Valeurs de country_risk avant encodage :
country_risk
low       6007
medium    2551
high      1442
Name: count, dtype: int64

✓ One-Hot Encoding appliqué sur 'country_risk'
Nouvelles colonnes créées : ['country_risk_high', 'country_risk_low', 'country_risk_medium']


,country_risk_high,country_risk_low,country_risk_medium
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,False,True


### 4.2 One-Hot Encoding pour device_type

In [6]:
# Afficher les valeurs uniques avant encodage
print("📊 Valeurs de device_type avant encodage :")
print(df_selected['device_type'].value_counts())

# Appliquer One-Hot Encoding sur device_type
device_type_encoded = pd.get_dummies(df_selected['device_type'], prefix='device_type', drop_first=False)

print(f"\n✓ One-Hot Encoding appliqué sur 'device_type'")
print(f"Nouvelles colonnes créées : {device_type_encoded.columns.tolist()}")

# Afficher un aperçu
device_type_encoded.head()

📊 Valeurs de device_type avant encodage :
device_type
mobile     5498
desktop    3483
tablet     1019
Name: count, dtype: int64

✓ One-Hot Encoding appliqué sur 'device_type'
Nouvelles colonnes créées : ['device_type_desktop', 'device_type_mobile', 'device_type_tablet']


,device_type_desktop,device_type_mobile,device_type_tablet
0,False,True,False
1,False,True,False
2,False,True,False
3,True,False,False
4,False,True,False


## 5. Assembler le dataset final

In [7]:
# Supprimer les colonnes catégorielles originales
df_final = df_selected.drop(['country_risk', 'device_type'], axis=1)

# Ajouter les colonnes encodées
df_final = pd.concat([df_final, country_risk_encoded, device_type_encoded], axis=1)

print(f"✓ Dataset final assemblé")
print(f"Dimensions : {df_final.shape[0]} lignes × {df_final.shape[1]} colonnes")
print(f"\nColonnes finales :")
for i, col in enumerate(df_final.columns, 1):
    print(f"  {i}. {col}")

✓ Dataset final assemblé
Dimensions : 10000 lignes × 11 colonnes

Colonnes finales :
  1. is_foreign_transaction
  2. account_age_days
  3. num_transactions_24h
  4. transaction_amount
  5. fraud
  6. country_risk_high
  7. country_risk_low
  8. country_risk_medium
  9. device_type_desktop
  10. device_type_mobile
  11. device_type_tablet


In [8]:
# Afficher les premières lignes du dataset final
print("📊 Aperçu du dataset final encodé :")
df_final.head(10)

📊 Aperçu du dataset final encodé :


,is_foreign_transaction,account_age_days,num_transactions_24h,transaction_amount,fraud,country_risk_high,country_risk_low,country_risk_medium,device_type_desktop,device_type_mobile,device_type_tablet
0,1,3615,3,15.01,0,False,True,False,False,True,False
1,0,209,2,24.67,0,False,True,False,False,True,False
2,0,438,4,92.79,0,False,True,False,False,True,False
3,1,1830,6,38.67,0,False,True,False,True,False,False
4,0,1686,3,69.14,0,False,False,True,False,True,False
5,1,997,4,52.55,0,False,True,False,False,True,False
6,0,1236,1,19.90,0,True,False,False,False,True,False
7,0,1114,4,51.09,0,False,True,False,False,True,False
8,1,318,4,25.71,0,True,False,False,True,False,False
9,1,3414,5,25.59,0,False,True,False,True,False,False


## 6. Vérification des types de données

In [9]:
# Vérifier les types de données
print("📋 Types de données finaux :")
print(df_final.dtypes)

# Vérifier qu'il n'y a plus de variables object (texte)
object_cols = df_final.select_dtypes(include=['object']).columns.tolist()
if len(object_cols) == 0:
    print("\n✓ Toutes les variables sont numériques (prêtes pour le ML)")
else:
    print(f"\n⚠️ Variables non-numériques restantes : {object_cols}")

📋 Types de données finaux :
is_foreign_transaction      int64
account_age_days            int64
num_transactions_24h        int64
transaction_amount        float64
fraud                       int64
country_risk_high            bool
country_risk_low             bool
country_risk_medium          bool
device_type_desktop          bool
device_type_mobile           bool
device_type_tablet           bool
dtype: object

✓ Toutes les variables sont numériques (prêtes pour le ML)


## 7. Séparation X (features) et Y (cible)

In [10]:
# Séparer les features (X) de la variable cible (Y)
X = df_final.drop('fraud', axis=1)  # Features
y = df_final['fraud']                # Cible

print("✓ Données séparées :")
print(f"  X (features) : {X.shape[0]} lignes × {X.shape[1]} colonnes")
print(f"  y (cible)    : {y.shape[0]} valeurs")

print(f"\n📊 Colonnes de X (features) :")
for i, col in enumerate(X.columns, 1):
    print(f"  {i}. {col}")

print(f"\n📊 Distribution de y (cible) :")
print(y.value_counts())
print(f"Pourcentage de fraudes : {(y.sum() / len(y) * 100):.2f}%")

✓ Données séparées :
  X (features) : 10000 lignes × 10 colonnes
  y (cible)    : 10000 valeurs

📊 Colonnes de X (features) :
  1. is_foreign_transaction
  2. account_age_days
  3. num_transactions_24h
  4. transaction_amount
  5. country_risk_high
  6. country_risk_low
  7. country_risk_medium
  8. device_type_desktop
  9. device_type_mobile
  10. device_type_tablet

📊 Distribution de y (cible) :
fraud
0    9449
1     551
Name: count, dtype: int64
Pourcentage de fraudes : 5.51%


## 8. Normalisation des features numériques (optionnel mais recommandé)

In [11]:
# Identifier les colonnes numériques originales (avant one-hot encoding)
numeric_features = [
    'is_foreign_transaction',
    'account_age_days',
    'num_transactions_24h',
    'transaction_amount'
]

# Créer une copie pour la normalisation
X_normalized = X.copy()

# Initialiser le scaler
scaler = StandardScaler()

# Normaliser uniquement les colonnes numériques (pas les one-hot encodées)
X_normalized[numeric_features] = scaler.fit_transform(X[numeric_features])

print("✓ Normalisation appliquée sur les features numériques")
print(f"Features normalisées : {numeric_features}")
print(f"\n📊 Statistiques après normalisation :")
print(X_normalized[numeric_features].describe())

✓ Normalisation appliquée sur les features numériques
Features normalisées : ['is_foreign_transaction', 'account_age_days', 'num_transactions_24h', 'transaction_amount']

📊 Statistiques après normalisation :
       is_foreign_transaction  account_age_days  num_transactions_24h  \
count            1.000000e+04      1.000000e+04          1.000000e+04   
mean             2.273737e-17     -7.318590e-17         -9.467982e-17   
std              1.000050e+00      1.000050e+00          1.000050e+00   
min             -5.844327e-01     -1.735096e+00         -1.588473e+00   
25%             -5.844327e-01     -8.673108e-01         -9.526770e-01   
50%             -5.844327e-01      1.862028e-03         -3.168808e-01   
75%              1.711061e+00      8.781450e-01          3.189154e-01   
max              1.711061e+00      1.731884e+00          4.769489e+00   

       transaction_amount  
count        1.000000e+04  
mean         3.215206e-17  
std          1.000050e+00  
min         -9.843178e

## 9. Sauvegarde des données préparées

In [12]:
# Sauvegarder le dataset complet encodé (sans normalisation)
df_final.to_csv('../data/processed/fraud_encoded.csv', index=False)
print("✓ Dataset encodé sauvegardé : data/processed/fraud_encoded.csv")

# Sauvegarder X et y séparément (sans normalisation)
X.to_csv('../data/processed/X_features.csv', index=False)
y.to_csv('../data/processed/y_target.csv', index=False)
print("✓ Features (X) sauvegardées : data/processed/X_features.csv")
print("✓ Cible (y) sauvegardée : data/processed/y_target.csv")

# Sauvegarder X normalisé et y
X_normalized.to_csv('../data/processed/X_features_normalized.csv', index=False)
print("✓ Features normalisées sauvegardées : data/processed/X_features_normalized.csv")

✓ Dataset encodé sauvegardé : data/processed/fraud_encoded.csv
✓ Features (X) sauvegardées : data/processed/X_features.csv
✓ Cible (y) sauvegardée : data/processed/y_target.csv
✓ Features normalisées sauvegardées : data/processed/X_features_normalized.csv


## ✅ Résumé du Feature Engineering

In [13]:
print("=" * 80)
print("RÉSUMÉ DU FEATURE ENGINEERING")
print("=" * 80)

print("\n1. FEATURES SÉLECTIONNÉES")
print("-" * 80)
print("✓ 6 features gardées sur 8 features originales")
print("✓ 2 features supprimées (transaction_hour, avg_amount_30d)")

print("\n2. ENCODAGE DES VARIABLES CATÉGORIELLES")
print("-" * 80)
print("✓ country_risk → One-Hot Encoding (3 colonnes)")
print("✓ device_type → One-Hot Encoding (3 colonnes)")

print("\n3. DATASET FINAL")
print("-" * 80)
print(f"✓ Dimensions : {X.shape[0]} lignes × {X.shape[1]} features")
print(f"✓ Variable cible : {y.shape[0]} valeurs")
print(f"✓ Toutes les variables sont numériques")

print("\n4. FICHIERS SAUVEGARDÉS")
print("-" * 80)
print("✓ fraud_encoded.csv (dataset complet)")
print("✓ X_features.csv (features sans normalisation)")
print("✓ X_features_normalized.csv (features normalisées)")
print("✓ y_target.csv (variable cible)")

print("\n5. PROCHAINES ÉTAPES")
print("-" * 80)
print("→ Notebook 04 : Split train/test")
print("→ Notebook 05 : Entraînement du modèle")
print("→ Notebook 06 : Évaluation des performances")

print("\n" + "=" * 80)

RÉSUMÉ DU FEATURE ENGINEERING

1. FEATURES SÉLECTIONNÉES
--------------------------------------------------------------------------------
✓ 6 features gardées sur 8 features originales
✓ 2 features supprimées (transaction_hour, avg_amount_30d)

2. ENCODAGE DES VARIABLES CATÉGORIELLES
--------------------------------------------------------------------------------
✓ country_risk → One-Hot Encoding (3 colonnes)
✓ device_type → One-Hot Encoding (3 colonnes)

3. DATASET FINAL
--------------------------------------------------------------------------------
✓ Dimensions : 10000 lignes × 10 features
✓ Variable cible : 10000 valeurs
✓ Toutes les variables sont numériques

4. FICHIERS SAUVEGARDÉS
--------------------------------------------------------------------------------
✓ fraud_encoded.csv (dataset complet)
✓ X_features.csv (features sans normalisation)
✓ X_features_normalized.csv (features normalisées)
✓ y_target.csv (variable cible)

5. PROCHAINES ÉTAPES
--------------------------------